## OpenAI Parallel Function calling: Query external SQLite DB

<img src="images/gpt_parallel_function_calling_db.png">

This script demonstrates how to use the OpenAI API to call a function that queries a database.
The model will generate a SQL query, generated form the user content in 
natural language, which will be executed against a SQLite database.

All this demonstrates how to use the OpenAI API to call a function that interacts with an external data source,
such as a database: SQLite, MySQL, PostgreSQL, etc.

This is a modified version of the script from the OpenAI API cookbook. Partly based on the example [here](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models)

To use this notebook, you must follow the following steps:

1. Install sqlite: `pip install sqlite3`
2. run `python customer_sqlite_db.py`. This will create a fake `customers.db` 


In [1]:
import os
import warnings
from dotenv import load_dotenv, find_dotenv
from typing import List
import openai
from openai import OpenAI
from tenacity import retry, stop_after_attempt, wait_random_exponential
from customer_db_utils import  execute_function_call, get_database_schema, connect_db
from termcolor import colored  

#### Define some utility functions

In [2]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(clnt:object, messages:object,
                             tools=None, tool_choice=None, 
                             model="gpt4-turbo-preview"):
    """
    Send a chat completion request using the OpenAI API."""
    try:
        response = clnt.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [3]:
def pretty_print_conversation(messages: List[dict]):
    """
    Print the conversation between the user, the assistant, and the function,
    each with a different color for readability.
    """
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

#### Load .env files for keys and models to use

In [4]:
_ = load_dotenv(find_dotenv()) # read local .env file
warnings.filterwarnings('ignore')
openai.api_base = os.getenv("ANYSCALE_API_BASE", os.getenv("OPENAI_API_BASE"))
openai.api_key = os.getenv("ANYSCALE_API_KEY", os.getenv("OPENAI_API_KEY"))
MODEL = os.getenv("MODEL")
print(f"Using MODEL={MODEL}; base={openai.api_base}")

Using MODEL=gpt-4-turbo-preview; base=https://api.openai.com/v1


Create an OpenAI client

In [5]:
client = OpenAI(
        api_key = openai.api_key,
        base_url = openai.api_base
)

### Step 0: Fetch the database schema

In [6]:
conn = connect_db("customers.db")
database_schema_string = get_database_schema(conn)
print(database_schema_string)

Table: customer_data
Columns: customer_name, product_name, date_of_purchase, price_paid, city


### Step 1: define the function to call for the model

In [7]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "query_customer_database",
            "description": "Use this function to answer user questions about fake customer database. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                         "type": "string",
                         "description": f"""
                                 SQL query extracting info to answer the user's question.
                                 SQL should be written using this database schema: {database_schema_string}
                                 The query should be returned in plain text, not in JSON.
                                 """,
                        }
                    },
                    "required": ["query"],
                },
            }
    }
]

### Step 2: define the messages to send to the model

The user content is expressed in natural language to query the database. This request will be converted into an SQL query and
returned back to the user.

In [8]:
messages = []
messages.append({"role": "system", 
                 "content": """
                     Answer user questions by generating SQL queries against the Customer Database.
                     SQL query extracting info to answer the user's question.
                     SQL should be written using this database schema:
                     {database_schema_string}
                     The query should be returned in plain text, not in JSON"""})
messages.append({"role": "user", 
                  "content": "Hi, List at most five customer names, city they live in, products they bought, prices paid for the most expensive product"})


### Step 3: Send the messages and function call info to the model
This request will generate a response (with an SQL generated by the model)

In [9]:
chat_response = chat_completion_request(client, messages, tools,
                                            tool_choice={"type": "function", 
                                                          "function": {"name": "query_customer_database"}},
                                            model=MODEL)
print(chat_response)

ChatCompletion(id='chatcmpl-8m5aqh3kVQbOUjoyTMBQZ8csHcCRV', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_vVUF2Iyb8qE5WIY3y9pBaEWT', function=Function(arguments='{"query":"SELECT customer_name, city, product_name, MAX(price_paid) AS price_paid FROM customer_data GROUP BY customer_name ORDER BY price_paid DESC LIMIT 5"}', name='query_customer_database'), type='function')]))], created=1706471936, model='gpt-4-0125-preview', object='chat.completion', system_fingerprint='fp_376b7f78b9', usage=CompletionUsage(completion_tokens=35, prompt_tokens=211, total_tokens=246))


In [10]:
# Extract the message returned by the model
assistant_message = chat_response.choices[0].message
print(assistant_message)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_vVUF2Iyb8qE5WIY3y9pBaEWT', function=Function(arguments='{"query":"SELECT customer_name, city, product_name, MAX(price_paid) AS price_paid FROM customer_data GROUP BY customer_name ORDER BY price_paid DESC LIMIT 5"}', name='query_customer_database'), type='function')])


In [11]:
# Extract the function call returned by the model
assistant_message.content = str(assistant_message.tool_calls[0].function)
print(assistant_message.content)

Function(arguments='{"query":"SELECT customer_name, city, product_name, MAX(price_paid) AS price_paid FROM customer_data GROUP BY customer_name ORDER BY price_paid DESC LIMIT 5"}', name='query_customer_database')


In [12]:
# Append the function call query generated by the model with the
# assistant role
messages.append({"role": assistant_message.role, "content": assistant_message.content})

In [13]:
# check if the model wanted to call a function
if assistant_message.tool_calls:
    # call the function with the query generated by the model
    results = execute_function_call(conn, assistant_message)
    messages.append({"role": "function", "tool_call_id": assistant_message.tool_calls[0].id, "name": assistant_message.tool_calls[0].function.name, "content": results})
    pretty_print_conversation(messages)

system: 
                     Answer user questions by generating SQL queries against the Customer Database.
                     SQL query extracting info to answer the user's question.
                     SQL should be written using this database schema:
                     {database_schema_string}
                     The query should be returned in plain text, not in JSON

user: Hi, List at most five customer names, city they live in, products they bought, prices paid for the most expensive product

assistant: Function(arguments='{"query":"SELECT customer_name, city, product_name, MAX(price_paid) AS price_paid FROM customer_data GROUP BY customer_name ORDER BY price_paid DESC LIMIT 5"}', name='query_customer_database')

function (query_customer_database): [('Virginia Clark', 'Aaronburgh', 'Laptop', 1000.0), ('Matthew Jacobs', 'Patricialand', 'Camera', 1000.0), ('Laura Brown', 'Port Denisemouth', 'Laptop', 1000.0), ('Kelli Vega', 'Port Lydiafort', 'Smartwatch', 1000.0), ('Kathryn 

### Step 4: Send more queries as messages to the model

In [14]:
messages = []
messages.append({"role": "user", 
                     "content": """List all customer name, city, the product they bought, and price they paid 
                     and who live in Port Leefort, Lake Phillipview, East Deanburgh, and East Shelleyside."""})

### Step 5: Send the messages and function call info to the model

In [15]:
chat_response = chat_completion_request(client, messages, tools,
                                            tool_choice={"type": "function", 
                                                          "function": {"name": "query_customer_database"}},
                                            model=MODEL)
print(chat_response)

ChatCompletion(id='chatcmpl-8m5atIf2UADAZKttPCqSZo4S3Km7M', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_01Cu9wWUPDMWVxtSewirA9Ks', function=Function(arguments='{"query":"SELECT customer_name, city, product_name, price_paid FROM customer_data WHERE city IN (\'Port Leefort\', \'Lake Phillipview\', \'East Deanburgh\', \'East Shelleyside\')"}', name='query_customer_database'), type='function')]))], created=1706471939, model='gpt-4-0125-preview', object='chat.completion', system_fingerprint='fp_376b7f78b9', usage=CompletionUsage(completion_tokens=44, prompt_tokens=172, total_tokens=216))


### Step 6: Get the messages returned by the model

In [16]:
assistant_message = chat_response.choices[0].message
assistant_message.content = str(assistant_message.tool_calls[0].function)
messages.append({"role": assistant_message.role, "content": assistant_message.content})
if assistant_message.tool_calls:
    results = execute_function_call(conn, assistant_message)
    messages.append({"role": "function", "tool_call_id": assistant_message.tool_calls[0].id, "name": assistant_message.tool_calls[0].function.name, "content": results})
    pretty_print_conversation(messages)

user: List all customer name, city, the product they bought, and price they paid 
                     and who live in Port Leefort, Lake Phillipview, East Deanburgh, and East Shelleyside.

assistant: Function(arguments='{"query":"SELECT customer_name, city, product_name, price_paid FROM customer_data WHERE city IN (\'Port Leefort\', \'Lake Phillipview\', \'East Deanburgh\', \'East Shelleyside\')"}', name='query_customer_database')

function (query_customer_database): [('Joseph Peterson', 'Port Leefort', 'Shorts', 155.94), ('Robert Rich', 'East Shelleyside', 'Pants', 647.51), ('Michael Sims', 'Lake Phillipview', 'Headphones', 43.32), ('Phyllis Moon', 'East Deanburgh', 'Laptop', 160.65), ('Matthew Estrada', 'Lake Phillipview', 'Laptop', 902.68)]



In [18]:
conn.close()